In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC

In [2]:
sub = pd.read_csv('sub_best.csv', index_col=0)
sup = pd.read_csv('sup_best.csv', index_col=0)

sub.head()

,5987ffb6ff306bfe127f6cae71d44a3a,8854d266763a7c79a5f4375403bb84cf,74707ec9eebf753ee133f984724b8d60,bff540f2d9cf8c25ed1bc1a5061f023c,1218c7b98d842dca062123cca16f33e4,f722761f392fab0a22b1882b323b4c42,394b7bd8355b485e02d10118bb1d82ca,7fb08689a17596b1f270584ded48755b,5fe32eb493b0033b691a1bd421b2250f,aeb1208d3560f1316d1987d687b231c9,...,401a1417aaf2de6a13f12a21b773a1e1,099b6193911807f83a40481806d29b28,76b6e42b4e14cb54000f3fc081d394b3,7182cda91061481d5dd6e9bacde8444a,5a31c9f5c0147655a9a9a5bab1b3ee2a,889f988e5e61b5fc199c249640cf282a,3694449ee1697514d252d8a500b756c4,4c0f8fe6e46eafa961142b0ae95aebe6,e0cc319b3897a1b188728cd99e618615,bec77cb21659cac2902f33ac539c74e9
P11-SubG-14,0.000000,0.000000,0.000000,0.054099,0.000000,0.063555,0.050708,0.000000,0.118106,0.008250,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
P11-SubG-21,0.000000,0.000000,0.000000,0.002343,0.001396,0.021237,0.035196,0.000000,0.035844,0.007478,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
P12-SubG-14,0.009949,0.000262,0.003334,0.003806,0.001102,0.009686,0.045727,0.016511,0.016406,0.002520,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
P12-SubG-21,0.000000,0.000000,0.001715,0.001858,0.000750,0.012184,0.019080,0.006503,0.028691,0.001929,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
P13-SubG-14,0.000000,0.000000,0.000000,0.043843,0.000000,0.008733,0.009969,0.003349,0.045312,0.003143,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
sub_x = sub.values
sup_x = sup.values
X = np.concatenate((sub_x, sup_x))
y = ['sub'] * sub_x.shape[0] + ['supra'] * sup_x.shape[0]
y = np.asarray(y)


In [6]:
from sklearn.model_selection import StratifiedKFold


model = SVC(kernel='linear')
skf = StratifiedKFold(n_splits=20)
scores = []
for train, test in skf.split(X, y):
    X_train, X_test = X[train], X[test]
    y_train, y_test = y[train], y[test]
    model.fit(X_train, y_train)
    scores.append(model.score(X[test],y[test]))

scores = np.asarray(scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.75 (+/- 0.45)


In [7]:
from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

from sklearn.feature_selection import SelectKBest

#X_new = SelectKBest(mutual_info_classif, k=200).fit_transform(X, y)


tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear', 'poly','sigmoid'], 'C': [1, 10, 100, 1000]}]
scores = ['accuracy']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(SVC(), tuned_parameters, cv=20,
                       scoring='%s' % score)
    clf.fit(X,y)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))

# Tuning hyper-parameters for accuracy

Best parameters set found on development set:

{'C': 10, 'kernel': 'linear'}

Grid scores on development set:

0.733 (+/-0.427) for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
0.733 (+/-0.427) for {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
0.733 (+/-0.427) for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
0.733 (+/-0.427) for {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
0.733 (+/-0.427) for {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
0.733 (+/-0.427) for {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}
0.733 (+/-0.427) for {'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}
0.733 (+/-0.427) for {'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}
0.733 (+/-0.427) for {'C': 1, 'kernel': 'linear'}
0.533 (+/-0.249) for {'C': 1, 'kernel': 'poly'}
0.733 (+/-0.427) for {'C': 1, 'kernel': 'sigmoid'}
0.783 (+/-0.423) for {'C': 10, 'kernel': 'linear'}
0.533 (+/-0.249) for {'C': 10, 'kernel': 'poly'}
0.733 (+/-0.427) for {'C': 10, 'kernel': 'sigmoid'}
0.767 (+/-0.464) for {'C': 1

In [14]:

warnings.warn = warn

results = []
for k in [100,200,300,400,500]:
    desc = "{0} features".format(k)
    X_new = SelectKBest(mutual_info_classif, k=k).fit_transform(X, y)


    tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                        {'kernel': ['linear', 'poly','sigmoid'], 'C': [1, 10, 100, 1000]}]
    scores = ['accuracy']

    for score in scores:
        print("# Tuning hyper-parameters for %s" % score)
        print()

        clf = GridSearchCV(SVC(), tuned_parameters, cv=20,
                       scoring='%s' % score)
        clf.fit(X_new,y)

        print("Best parameters set found on development set: %s" % desc)
        print()
        print(clf.best_params_)
        
        print()
        print("Grid scores on development set:")
        print()
        means = clf.cv_results_['mean_test_score']
        stds = clf.cv_results_['std_test_score']
        for mean, std, params in zip(means, stds, clf.cv_results_['params']):
            print("%0.3f (+/-%0.03f) for %r"
                  % (mean, std * 2, params))

# Tuning hyper-parameters for accuracy

Best parameters set found on development set: 100 features

{'C': 1000, 'kernel': 'linear'}

Grid scores on development set:

0.617 (+/-0.382) for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
0.617 (+/-0.382) for {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
0.617 (+/-0.382) for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
0.617 (+/-0.382) for {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
0.617 (+/-0.382) for {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
0.617 (+/-0.382) for {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}
0.617 (+/-0.382) for {'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}
0.617 (+/-0.382) for {'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}
0.617 (+/-0.382) for {'C': 1, 'kernel': 'linear'}
0.517 (+/-0.180) for {'C': 1, 'kernel': 'poly'}
0.617 (+/-0.382) for {'C': 1, 'kernel': 'sigmoid'}
0.617 (+/-0.382) for {'C': 10, 'kernel': 'linear'}
0.517 (+/-0.180) for {'C': 10, 'kernel': 'poly'}
0.617 (+/-0.382) for {'C': 10, 'kernel': 'sigmoid'}
0.900 (+/-0.3

In [15]:
for k in [600,700,800,900,1000]:
    desc = "{0} features".format(k)
    X_new = SelectKBest(mutual_info_classif, k=k).fit_transform(X, y)


    tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                        {'kernel': ['linear', 'poly','sigmoid'], 'C': [1, 10, 100, 1000]}]
    scores = ['accuracy']

    for score in scores:
        print("# Tuning hyper-parameters for %s" % score)
        print()

        clf = GridSearchCV(SVC(), tuned_parameters, cv=20,
                       scoring='%s' % score)
        clf.fit(X_new,y)

        print("Best parameters set found on development set: %s" % desc)
        print()
        print(clf.best_params_)
        
        print()
        print("Grid scores on development set:")
        print()
        means = clf.cv_results_['mean_test_score']
        stds = clf.cv_results_['std_test_score']
        for mean, std, params in zip(means, stds, clf.cv_results_['params']):
            print("%0.3f (+/-%0.03f) for %r"
                  % (mean, std * 2, params))

# Tuning hyper-parameters for accuracy

Best parameters set found on development set: 600 features

{'C': 100, 'kernel': 'linear'}

Grid scores on development set:

0.667 (+/-0.453) for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
0.667 (+/-0.453) for {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
0.667 (+/-0.453) for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
0.667 (+/-0.453) for {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
0.667 (+/-0.453) for {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
0.667 (+/-0.453) for {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}
0.667 (+/-0.453) for {'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}
0.667 (+/-0.453) for {'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}
0.667 (+/-0.453) for {'C': 1, 'kernel': 'linear'}
0.517 (+/-0.180) for {'C': 1, 'kernel': 'poly'}
0.667 (+/-0.453) for {'C': 1, 'kernel': 'sigmoid'}
0.667 (+/-0.453) for {'C': 10, 'kernel': 'linear'}
0.517 (+/-0.180) for {'C': 10, 'kernel': 'poly'}
0.667 (+/-0.453) for {'C': 10, 'kernel': 'sigmoid'}
0.817 (+/-0.42

In [8]:
from keras.layers import Lambda, Input, Dense, MaxPooling1D, BatchNormalization
from keras.models import Model
from keras.losses import mse, binary_crossentropy
from keras import utils
from keras import backend as K
from keras import layers
from keras import objectives
from keras import optimizers
from keras import regularizers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

Using TensorFlow backend.


In [9]:
def create_model():
    inputs = layers.Input(shape=(100,1,))
    x = layers.Conv1D(32, 2)(inputs)
    x = layers.MaxPooling1D()(x)
    x = layers.Dropout(0.2)(x)
    x = BatchNormalization()(x)
    x = layers.Flatten()(x)
    x = layers.Dense(30, activation='relu')(x)
    outputs = layers.Dense(2,activation='softmax')(x)
    fancy_model = Model(inputs=inputs, outputs=outputs)

    fancy_model.summary()
    fancy_model.compile(loss='binary_crossentropy',
                   optimizer='Adam', metrics=['accuracy'])
    return fancy_model

In [10]:
X_new = SelectKBest(mutual_info_classif, k=100).fit_transform(X, y)
print(X_new.shape)
X_new = np.reshape(X_new, (60,100,1))

y = [0] * 30 + [1] * 30
y = np.asarray(y)

x_train, x_test, y_train, y_test = train_test_split(X_new, y, test_size=0.3)

enc = OneHotEncoder(sparse=False)

labels = np.arange(2)
labels = np.reshape(labels, (len(labels),1))
enc.fit(labels)
y_train = enc.transform(np.reshape(y_train, (-1,1)))
y_test = enc.transform(np.reshape(y_test, (-1,1)))



(60, 100)


In [12]:
def gaussian_expansion(x_data,y, magnitude):
    mu, sigma = 0, 0.01
    num_samples = x_data.shape[0]
    x_noised = []
    for i in range(magnitude):
        noise = np.random.normal(mu, sigma, x_data.shape)
        x_noised.append(x_data + noise)
        
    y_new = [y] * (magnitude +1)
    y_new = np.asarray(y_new).flatten()
    print(y_new.shape)
    x_noised = np.asarray(x_noised)
    x_noised = x_noised.reshape(magnitude*x_data.shape[0],100,1)
    
    y_new = enc.transform(np.reshape(y_new, (-1,1)))
    new_x = np.concatenate((x_data, x_noised))
    
    print(y_new.shape)
    print(new_x.shape)
    return(new_x, y_new)
    
X_new = SelectKBest(mutual_info_classif, k=100).fit_transform(X, y)
X_new = np.reshape(X_new, (60,100,1))
y = [0] * 30 + [1] * 30
y = np.asarray(y)

enc = OneHotEncoder(sparse=False)
labels = np.arange(2)
labels = np.reshape(labels, (len(labels),1))
enc.fit(labels)
#y = enc.transform(np.reshape(y, (-1,1)))

skf = StratifiedKFold(n_splits=5)
scores = []
for train, test in skf.split(X_new, y):
    X_train, X_test = X_new[train], X_new[test]
    y_train, y_test = y[train], y[test]
    y_test = enc.transform(np.reshape(y_test, (-1,1)))
    x_tr, y_tr = gaussian_expansion(X_train, y_train, 20)
    model = create_model()
    model.fit(x_tr, y_tr, batch_size=6, epochs=5, validation_data=(X_test, y_test))
    evals = model.evaluate(X_test, y_test)
    scores.append(evals)
    
    

(1008,)
(1008, 2)
(1008, 100, 1)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100, 1)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 99, 32)            96        
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 49, 32)            0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 49, 32)            0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 49, 32)            128       
_________________________________________________________________
flatten_1 (Flatten)          (None, 1568)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 30)    

KeyboardInterrupt: 

In [41]:
acc = []
for i in scores:
    acc.append(i[1])
acc = np.asarray(acc)
acc.mean()

0.8111111164093018